In [2]:
import pandas as pd
from glob import glob
from os import path


In [27]:
def load(indir):
    dfs = []
    for infile in sorted(glob(f"{indir}/*.csv")):
        base = path.splitext(path.basename(infile))[0]
        if base == "index":
            continue
        dfs.append(pd.read_csv(infile, dtype=str, na_filter=False).assign(file=base))
    return pd.concat(dfs)

norm = load("AESI-norm")
norm

,sab,code,str,cui,concept_name,tags,comments,file,review_author_0,review_date_0,review_content_0,review_author_1,review_date_1,review_content_1,review_author_2,review_date_2,review_content_2,review_author_3,review_date_3,review_content_3
0,ICD10,I74,Arterial embolism and thrombosis,C0155749,Arterial embolus and thrombosis,Narrow,,B_ARTERIALEMBOLISM_AESI_arterial embolism,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ICD10,I74.0,Embolism and thrombosis of abdominal aorta,,,Narrow,,B_ARTERIALEMBOLISM_AESI_arterial embolism,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ICD10,I74.1,Embolism and thrombosis of other and unspecifi...,,,Narrow,,B_ARTERIALEMBOLISM_AESI_arterial embolism,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ICD10,I74.2,Embolism and thrombosis of arteries of upper e...,,,Narrow,,B_ARTERIALEMBOLISM_AESI_arterial embolism,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ICD10,I74.3,Embolism and thrombosis of arteries of lower e...,,,Narrow,,B_ARTERIALEMBOLISM_AESI_arterial embolism,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
787,SNOMEDCT_US,698247007,Other cardiac dysrhythmia NOS,,,Narrow,,C_ARRH_AESI_Arrythmia all,Joan/Cristina,2022-02-01 00:00:00,,Joan Fortuny,2022-12-12 00:00:00,no change,sima,2023-02-01 00:00:00,no change,NaN,NaN,NaN
788,SNOMEDCT_US,698247007,Other cardiac dysrhythmias,,,Narrow,,C_ARRH_AESI_Arrythmia all,Joan/Cristina,2022-02-01 00:00:00,,Joan Fortuny,2022-12-12 00:00:00,no change,sima,2023-02-01 00:00:00,no change,NaN,NaN,NaN
789,SNOMEDCT_US,698247007,Cardiac arrhythmia,C0003811,Cardiac Arrhythmia,Narrow,,C_ARRH_AESI_Arrythmia all,,,,Joan Fortuny,2022-12-12 00:00:00,no change,sima,2023-02-01 00:00:00,no change,NaN,NaN,NaN
790,SNOMEDCT_US,720448006,Typical atrial flutter,C3264370,Typical atrial flutter,Narrow,,C_ARRH_AESI_Arrythmia all,,,,Joan Fortuny,2022-12-12 00:00:00,no change,sima,2023-02-01 00:00:00,no change,NaN,NaN,NaN


In [103]:
dedup['sab code dedup_result'.split()].drop_duplicates().dedup_result.value_counts()

EXACT                    1562
BY_CODE_EQUIV             379
NONE                      204
BY_CODE                   174
CHANGE_CODING_SYSTEM        9
NONE_NO_CODING_SYSTEM       9
BY_NAME                     4
CODE_BY_CUI                 1
Name: dedup_result, dtype: int64

In [109]:
dedup_none = dedup[dedup.dedup_result == "NONE"]['cui sab code str'.split()]
dedup_none.sab.value_counts()

MEDCODEID    157
ICD10DA      128
ICPC           6
ICD10          2
SCTSPA         1
ICPC2P         1
Name: sab, dtype: int64

In [110]:
dedup_none[~dedup_none.sab.isin('MEDCODEID ICD10DA'.split())]

,cui,sab,code,str
132,,ICPC,B83.01,Hemophilia A/B
133,,ICPC,B83.02,Idiopathic thrombocytopenic purpura (ITP)
134,,ICPC,B83.06,Increased coagulation tendency
132,,ICPC,B83.01,Hemophilia A/B
133,,ICPC,B83.02,Idiopathic thrombocytopenic purpura (ITP)
134,,ICPC,B83.06,Increased coagulation tendency
127,,SCTSPA,10752381000119100,Fetal thrombocytopenia
3,,ICD10,D57.01,Sickle-cell anaemia without crisis
22,,ICPC2P,B83,
36,,ICD10,I48.9,"Atrial fibrillation and atrial flutter, unspec..."


In [ ]:
10752381000119100
10752381000119101

In [7]:
norm[['file', 'coding_system']].drop_duplicates().coding_system.value_counts()

ICD9CM         29
ICD10CM        29
ICD10          28
SNOMEDCT_US    28
SCTSPA         27
ICPC2P         25
MEDCODEID      25
RCD2           25
ICD10DA        23
ICPC           17
Free_text      16
MTHICD9        11
SNM             7
RCD             5
SNMI            1
p               1
ICD10/CM        1
e               1
                1
Name: coding_system, dtype: int64

In [10]:
norm[norm.coding_system.isin(["p", "e", ""])][['file', 'coding_system']].drop_duplicates()

,file,coding_system
227,B_COAGDEF_AESI_Coagulation deficiencies,e
261,B_COAGDEF_AESI_Coagulation deficiencies,p
1573,C_CARDIOMYOPATHY_COV_Cardiomyopathy excl. myo ...,


## Coding systems

In [6]:
def coding_systems(indir):
    res = set()
    for infile in sorted(glob(f"{indir}/*.csv")):
        if infile == f"{indir}/index.csv":
            continue
        df = pd.read_csv(infile, dtype=str, na_filter=False)
        for sab in df.coding_system:
            if sab:
                res.add(sab)
    return sorted(res)

for sab in coding_systems("AESI-norm"):
    print(f"- {sab}")

- CVB
- EXCLU
- Free_text
- HCPCS
- HCPT
- HSD_ACCERT
- IC9CM
- ICD-10-CM
- ICD-10-PCS
- ICD-9-CM
- ICD-9CM
- ICD10
- ICD10-PCS
- ICD10/CM
- ICD10CM
- ICD10DA
- ICD9-CM
- ICD9CM
- ICD9CM_HSD
- ICPC
- ICPC2
- ICPC2EENG
- ICPC2P
- MDR
- MEDCODEID
- MTHICD9
- NCMP-NCSP-NCRP
- PROCDA
- RCD
- RCD2
- SCTSPA
- SCTSPA_SNS
- SNM
- SNMI
- SNOMED
- SNOMEDCT_US
- SNOMED_US
- SPA_EXT
- ZA


echo 'select state from case_definitions ' \
| sudo -u postgres psql codemapper \
| head -n-1 | tail -n+3 \
| jq '.codingSystems' \
| grep '^  "' | sed 's/^  "\([^"]*\)",*$/\1/' \
| sort | uniq -c | sort -h

      1 ICPCNOR
      1 KCD5
      1 RCDAE
      1 WHOSPA
      2 CCSR_ICD10PCS
      2 ICPC2ICD10DUT
      2 ICPC2ICD10ENG
      2 LNC
      3 ICPCDUT
      4 SNMI
      9 HCPT
     10 HCPCS
     15 ICD10PCS
     15 ICPC2EDUT
     27 MSH
     65 SNM
     76 MDR
    189 ICD10/CM
    227 RCD
    326 ICPC2EENG
    364 ICPC
    416 SCTSPA
    446 MTHICD9
    501 SNOMEDCT_US
    583 ICPC2P
    607 ICD10
    623 RCD2
    632 ICD10CM
    731 ICD9CM


In [85]:
col_names = {"coding_system": "sab", "term": "str", "code_name": "str", "tag": "tags"}

final_v0 = load("VAC4EU-v0/AESI-final").rename(col_names, axis=1)
final = load("AESI-final").rename(col_names, axis=1)

cols = "sab code".split()
diff = pd.merge(
    final_v0.assign(tags=lambda df: df.tags.str.lower())[cols].drop_duplicates(),
    final.assign(tags=lambda df: df.tags.str.lower()).rename(col_names, axis=1)[cols].drop_duplicates(),
    on=["sab", "code"],
    how="outer",
    suffixes=("_v0", ""),
    indicator=True
)
print("ONLY in v0", (diff._merge == "left_only").sum())
print("NOT in v0", (diff._merge == "right_only").sum())
#changed_tags = (~pd.isna(diff.tags) | ~pd.isna(diff.tags_v0)) & (diff.tags != diff.tags_v0)
#print("CHANGED TAGS", changed_tags.sum())
diff[diff._merge == "left_only"]

ONLY in v0 13
NOT in v0 438


,sab,code,_merge
1553,ICPC2EENG,K05,left_only
1752,SNOMEDCT_US,251120003,left_only
1753,SNOMEDCT_US,251123001,left_only
1754,SNOMEDCT_US,251124007,left_only
1755,SNOMEDCT_US,276796006,left_only
1757,SNOMEDCT_US,29320008,left_only
1758,SNOMEDCT_US,314208002,left_only
1759,SNOMEDCT_US,421869004,left_only
1760,SNOMEDCT_US,425615007,left_only
1761,SNOMEDCT_US,427393009,left_only


In [98]:
dedup_v0.query('dedup_code == "K05"')["sab code str dedup_result dedup_coding_system dedup_code dedup_code_name".split()]

,sab,code,str,dedup_result,dedup_coding_system,dedup_code,dedup_code_name
288,ICPC,K05,Irregular heartbeat other,CHANGE_CODING_SYSTEM,ICPC2EENG,K05,Irregular heartbeat other


In [100]:
dedup.query('dedup_code == "K05"')["sab code str dedup_result dedup_sab dedup_code dedup_str".split()]

,sab,code,str,dedup_result,dedup_sab,dedup_code,dedup_str
288,ICPC,K05,Irregular heartbeat other,BY_CODE_EQUIV,ICPC,K05,Other abn/irreg heartbeat/pulse


In [71]:
dedup.drop([c for c in dedup.columns if c.startswith('review_')], axis=1).query("code == 'D68.5'")

,sab,code,str,cui,concept_name,tags,comments,dedup_result,dedup_comments,dedup_code,dedup_str,dedup_sab,dedup_cui,dedup_ttys,dedup_ignore,file
9,ICD10,D68.5,Primary thrombophilia,,,exclude,,CHANGE_CODING_SYSTEM,-,D68.5,Primary thrombophilia,ICD10CM,C2584620,"AB,HT",false,B_COAGDEF_AESI_Coagulation deficiencies
76,ICD10DA,D68.5,Primær trombofili,,,exclude,,NONE,-,-,-,-,-,-,-,B_COAGDEF_AESI_Coagulation deficiencies
9,ICD10,D68.5,Primary thrombophilia,,,narrow,,CHANGE_CODING_SYSTEM,-,D68.5,Primary thrombophilia,ICD10CM,C2584620,"AB,HT",false,B_COAGDEF_COV_Coagulation deficiencies
76,ICD10DA,D68.5,Primær trombofili,,,narrow,,NONE,-,-,-,-,-,-,-,B_COAGDEF_COV_Coagulation deficiencies


In [72]:
dedup_v0.drop([c for c in dedup_v0.columns if c.startswith('review_')], axis=1).query("code == 'D68.5'")

,coding_system,code,code_name,concept,concept_name,tags,comments,dedup_result,dedup_comment,dedup_code,...,dedup_coding_system,dedup_concept,dedup_changed,dedup_ttys,dedup_ignore,dedup_names_by_code,dedup_codes_by_name,dedup_original_code,dedup_original_code_name,file
9,ICD10,D68.5,Primary thrombophilia,,,exclude,,CHANGE_CODING_SYSTEM,-,D68.5,...,ICD10CM,C2584620,changed coding system from ICD10|cui from -,"AB,HT",false,-,-,D68.5,Primary thrombophilia,B_COAGDEF_AESI_Coagulation deficiencies
76,ICD10DA,D68.5,Primær trombofili,,,exclude,,NONE,-,-,...,-,-,-,-,-,-,D685,D68.5,Primær trombofili,B_COAGDEF_AESI_Coagulation deficiencies
9,ICD10,D68.5,Primary thrombophilia,,,narrow,,CHANGE_CODING_SYSTEM,-,D68.5,...,ICD10CM,C2584620,changed coding system from ICD10|cui from -,"AB,HT",false,-,-,D68.5,Primary thrombophilia,B_COAGDEF_COV_Coagulation deficiencies
76,ICD10DA,D68.5,Primær trombofili,,,narrow,,NONE,-,-,...,-,-,-,-,-,-,D685,D68.5,Primær trombofili,B_COAGDEF_COV_Coagulation deficiencies


In [49]:
diff[changed_tags]

,sab,code,str,tags,str_v0,tags_v0,_merge
9,ICD10CM,I66,embolism of cerebral artery,narrow,NaN,NaN,left_only
61,MTHICD9,444.22,"Arterial embolism or thrombosis, femoral",narrow,NaN,NaN,left_only
67,ICD10,D66,Hereditary factor VIII deficiency,exclude,Hereditary factor VIII deficiency,narrow,both
68,ICD10,D66,Hereditary factor VIII deficiency,narrow,Hereditary factor VIII deficiency,exclude,both
71,ICD10,D67,Hereditary factor IX deficiency,exclude,Hereditary factor IX deficiency,narrow,both
...,...,...,...,...,...,...,...
4285,SNOMEDCT_US,425615007,NaN,NaN,Chronic atrial flutter (disorder),narrow,right_only
4286,SNOMEDCT_US,427393009,NaN,NaN,Electrocardiogram: sinus arrhythmia (finding),narrow,right_only
4287,SNOMEDCT_US,49710005,NaN,NaN,Sinus bradycardia (disorder),narrow,right_only
4288,SNOMEDCT_US,733534002,NaN,NaN,Electrocardiographic complete left bundle bran...,narrow,right_only


In [86]:
dedup = load("AESI-dedup").rename(col_names, axis=1)
dedup_v0 = load("VAC4EU-v0/AESI-dedup").rename(col_names, axis=1)
dedup

,sab,code,str,cui,concept_name,tags,comments,dedup_result,dedup_comments,dedup_code,...,review_content_0,review_author_1,review_date_1,review_content_1,review_author_2,review_date_2,review_content_2,review_author_3,review_date_3,review_content_3
0,ICD10,I74,Arterial embolism and thrombosis,C0155749,Arterial embolus and thrombosis,Narrow,,EXACT,-,I74,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ICD10,I74.0,Embolism and thrombosis of abdominal aorta,,,Narrow,,EXACT,-,I74.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ICD10,I74.1,Embolism and thrombosis of other and unspecifi...,,,Narrow,,EXACT,-,I74.1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ICD10,I74.2,Embolism and thrombosis of arteries of upper e...,,,Narrow,,EXACT,-,I74.2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ICD10,I74.3,Embolism and thrombosis of arteries of lower e...,,,Narrow,,EXACT,-,I74.3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
787,SNOMEDCT_US,698247007,Other cardiac dysrhythmia NOS,,,Narrow,,BY_CODE,-,698247007,...,,Joan Fortuny,2022-12-12 00:00:00,no change,sima,2023-02-01 00:00:00,no change,NaN,NaN,NaN
788,SNOMEDCT_US,698247007,Other cardiac dysrhythmias,,,Narrow,,BY_CODE,-,698247007,...,,Joan Fortuny,2022-12-12 00:00:00,no change,sima,2023-02-01 00:00:00,no change,NaN,NaN,NaN
789,SNOMEDCT_US,698247007,Cardiac arrhythmia,C0003811,Cardiac Arrhythmia,Narrow,,EXACT,-,698247007,...,,Joan Fortuny,2022-12-12 00:00:00,no change,sima,2023-02-01 00:00:00,no change,NaN,NaN,NaN
790,SNOMEDCT_US,720448006,Typical atrial flutter,C3264370,Typical atrial flutter,Narrow,,EXACT,-,720448006,...,,Joan Fortuny,2022-12-12 00:00:00,no change,sima,2023-02-01 00:00:00,no change,NaN,NaN,NaN
